<a href="https://colab.research.google.com/github/FabioSicca/Organizacion-de-datos/blob/main/Spark3_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

20) Cantidad de Stubs por categoría en la Wikipedia. (⭐⭐). (Dado que hubo problemas para encontrar los stubs, en la clase de consulta se pidió finalmente que se calcule la cantidad de subcategorias por categoria).

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=8d867042161bd15248a18ba7d1559c8e8c73f1716d7359e79456b62cc87db26e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 36.5 MB of archives.
After this operatio

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
type(sc)

pyspark.context.SparkContext

In [ ]:
archivo = spark.read.csv("/content/drive/MyDrive/Wikipedia dataset/categorylinks.csv", header=True, inferSchema=True, multiLine=True, escape='"')

In [ ]:
archivo.show(10)

+-------+--------------------+--------------------+-------------------+--------------------+------------+-------+
|cl_from|               cl_to|          cl_sortkey|       cl_timestamp|   cl_sortkey_prefix|cl_collation|cl_type|
+-------+--------------------+--------------------+-------------------+--------------------+------------+-------+
|      5|Wikipedia:Manteni...|*LP:.R@FNNF@:.:P...|2018-06-06 21:42:29|Articulos solicit...| uca-es-u-kn|   page|
|      5|Wikipedia:PÃ¡gina...|*LP:.R@FNNF@:.:P...|2021-04-01 03:18:11|                null| uca-es-u-kn|   page|
|      7|             Andorra|  *D0FLL*\nÄÜ\n|2013-01-12 19:07:41|                    | uca-es-u-kn|   page|
|      7|Instituciones_pat...|      *D0FLL*\vÜ\n|2011-03-30 19:52:38|                null| uca-es-u-kn|   page|
|      7|Wikipedia:ArtÃ­cu...|      *D0FLL*\vÜ\n|2021-06-16 09:38:24|                null| uca-es-u-kn|   page|
|      7|Wikipedia:ArtÃ­cu...|      *D0FLL*\vÜ\n|2021-06-16 09:38:24|                n

In [ ]:
categorias = archivo.rdd

A continuación veo qué contienen los registros:

In [ ]:
categorias.take(10)

[Row(cl_from=5, cl_to='Wikipedia:Mantenimiento', cl_sortkey='*LP:.R@FN\x04NF@:.:P*0FN\x03\x06*LP:.R@FN\x04NF@:.:P*0FN\x01,\x88\x15\x01Ü±Ü\x19', cl_timestamp=datetime.datetime(2018, 6, 6, 21, 42, 29), cl_sortkey_prefix='Articulos solicitados', cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=5, cl_to='Wikipedia:PÃ¡ginas_con_enlaces_mÃ¡gicos_de_ISBN', cl_sortkey='*LP:.R@FN\x04NF@:.:P*0FN\x01B\x88\x15\x01Ü\x19', cl_timestamp=datetime.datetime(2021, 4, 1, 3, 18, 11), cl_sortkey_prefix=None, cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=7, cl_to='Andorra', cl_sortkey='\x04\x03\x06*D0FLL*\x01\n\x01ÄÜ\n', cl_timestamp=datetime.datetime(2013, 1, 12, 19, 7, 41), cl_sortkey_prefix=' ', cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=7, cl_to='Instituciones_patrocinadoras_de_equipos_ciclistas', cl_sortkey='*D0FLL*\x01\x0b\x01Ü\n', cl_timestamp=datetime.datetime(2011, 3, 30, 19, 52, 38), cl_sortkey_prefix=None, cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_fro

Filtro los registros entre aquellos que agregaron una página (no me sirven) de aquellos que agregaron una subcategoría a la categoría, luego los convierto en registros de tipo diccionario o (clave, valor) para usar reduceByKey y contar la cantidad de subcategorías para cada categoría.

In [ ]:
categorias.filter(lambda x: x.cl_type == "subcat").map(lambda x: (x.cl_to, 1)).reduceByKey(lambda a,b: a+b).collect() #.take(20) para ver un par de resultados

[('AntropologÃ\xada_cultural', 22),
 ('Espiritualidad', 35),
 ('Formas_de_cultura', 26),
 ('Historias_tradicionales', 1),
 ('Literatura_por_gÃ©nero', 17),
 ('ReligiÃ³n', 19),
 ('Wikipedia:CategorÃ\xadas_muy_amplias', 1338),
 ('Cultura_del_Antiguo_Egipto', 12),
 ('MitologÃ\xada_de_Asia', 14),
 ('MitologÃ\xada_de_Ã\x81frica', 14),
 ('MitologÃ\xada_del_Antiguo_Oriente_PrÃ³ximo', 5),
 ('MitologÃ\xada_por_cultura', 49),
 ('ReligiÃ³n_del_Antiguo_Egipto', 11),
 ('Disciplinas_de_la_biologÃ\xada', 47),
 ('Estructura', 15),
 ('Medicina', 46),
 ('Laureados_con_el_Premio_Nobel', 9),
 ('Premio_Nobel', 9),
 ('Premios_de_medicina', 4),
 ('InformÃ¡tica', 30),
 ('Obras_intelectuales', 13),
 ('Ciencias_aplicadas_por_disciplina', 50),
 ('Ciencias_de_la_salud', 14),
 ('Actividades_tabÃº', 3),
 ('Relaciones_Ã\xadntimas', 5),
 ('ReproducciÃ³n_humana', 10),
 ('TÃ©rminos_mÃ©dicos', 10),
 ('Religiones', 37),
 ('Religiones_abrahÃ¡micas', 11),
 ('Deportes_OlÃ\xadmpicos_de_Verano', 37),
 ('Deportes_de_equipo', 41

Vemos que entre los primeros se encuentra la categoría Medicina con 46 subcategorías, y si visitamos https://es.wikipedia.org/wiki/Categor%C3%ADa:Medicina se observa que efectivamente tiene 46 subcategorías